In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from pdll import PairwiseDifferenceClassifier

In [2]:
data = pd.read_csv('./combined_data.csv')

In [3]:
display(data.head())

,Cell_Type,Phase Volume,Phase_Variance,Phase_Surface_Area_to_Volume_Ratio,Phase_Skewness,Dry_Mass,Phase_Sphericity_Index,Phase_Surface_Area_to_Dry_Mass_Ratio,Mean_OPD,Projected_Area_To_Volume,Phase_Kurtosis,Energy,Median_OPD,Phase_Surface_Area,Density
0,Hek,3309.074158,57.654616,0.403143,0.342372,1.654537e+04,0.804988,0.080629,47.957596,0.020852,2.248171,3.920514e+03,47.083150,1334.030093,239.787982
1,Hek,3630.107691,923.235217,1.130838,-0.288402,1.815054e+04,0.278255,0.226168,46.539842,0.021487,1.790718,7.108911e+04,53.856019,4105.065431,232.699211
2,Hek,38236.014425,14237.096896,3.787462,1.703172,1.911801e+05,0.037900,0.757492,39.995831,0.025003,5.769890,1.359643e+07,0.000000,144817.461365,199.979155
3,Hek,52.984288,663.753975,209.137328,-0.172664,2.649214e+02,0.006157,41.827466,0.210255,4.756127,1.972272,1.666022e+05,0.000000,11080.992327,1.051276
4,Hek,224270.162358,7023.222553,1.455073,0.285291,1.121351e+06,0.054703,0.291015,58.817247,0.017002,3.030987,2.677252e+07,41.627913,326329.532081,294.086234


In [4]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
print(data.isnull().sum())

Cell_Type                                0
Phase Volume                             0
Phase_Variance                          48
Phase_Surface_Area_to_Volume_Ratio      22
Phase_Skewness                          48
Dry_Mass                                 0
Phase_Sphericity_Index                  65
Phase_Surface_Area_to_Dry_Mass_Ratio    22
Mean_OPD                                 0
Projected_Area_To_Volume                22
Phase_Kurtosis                          48
Energy                                   0
Median_OPD                               0
Phase_Surface_Area                       0
Density                                  0
dtype: int64


In [6]:
data = data.dropna()

In [7]:
print(data.isnull().sum())

Cell_Type                               0
Phase Volume                            0
Phase_Variance                          0
Phase_Surface_Area_to_Volume_Ratio      0
Phase_Skewness                          0
Dry_Mass                                0
Phase_Sphericity_Index                  0
Phase_Surface_Area_to_Dry_Mass_Ratio    0
Mean_OPD                                0
Projected_Area_To_Volume                0
Phase_Kurtosis                          0
Energy                                  0
Median_OPD                              0
Phase_Surface_Area                      0
Density                                 0
dtype: int64


In [8]:
X = data.drop(columns=['Cell_Type'])
y = data['Cell_Type'].map({'Hek': 0, 'A375': 1})  # Encoding classes as 0 and 1

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
def create_pairs(X, y):
    pairs = []
    labels = []
    
    for i in range(len(X)):
        for j in range(i + 1, len(X)):
            pairs.append(np.concatenate([X[i], X[j]]))
            labels.append(1 if y[i] == y[j] else 0)
    
    return np.array(pairs), np.array(labels)

In [11]:
X_train.shape


(10160, 14)

In [12]:
pairs_train, labels_train = create_pairs(X_train.values, y_train.values)
base_classifier = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)
base_classifier.fit(pairs_train, labels_train)